In [1]:
indels ='/Users/bencekover/Library/CloudStorage/OneDrive-UniversityCollegeLondon/MSci Bahler lab/S.-Pombe-biofilm/external data/Jeffares/Jeffares SNPs/ERZ289254/Spombe.indels.unique.nr57.final.annotated.2014-01-23.cleaned4.vcf.gz'
snps = '/Users/bencekover/Library/CloudStorage/OneDrive-UniversityCollegeLondon/MSci Bahler lab/S.-Pombe-biofilm/external data/Jeffares/Jeffares SNPs/ERZ289255/Spombe.2013-01-02.filt3c.nr57-final.snps.anno-snpeff3.cleaned4.vcf'
phenotype = "/Users/bencekover/Library/CloudStorage/OneDrive-UniversityCollegeLondon/MSci Bahler lab/S.-Pombe-biofilm/Bence folder/Phenotypes/phenotype_gwas.csv"
import hail as hl
from hail.plot import show
from pprint import pprint
hl.plot.output_notebook()
import pandas as pd


Loading BokehJS ...

In [2]:
hl.init()
contigs = ["I","II","III", "MT"]
lengths = {"I":5579133,"II":4539804,"III": 2452883, "MT": 19431}
my_reference_genome = hl.ReferenceGenome("pombe",contigs, lengths)

2022-11-16 18:44:48 WARN  NativeCodeLoader:60 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


2022-11-16 18:44:51 WARN  Utils:69 - Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


Running on Apache Spark version 3.1.3
SparkUI available at http://eduroam-int-dhcp-97-252-170.ucl.ac.uk:4041
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.104-1940d9e8eaab
LOGGING: writing to /Users/bencekover/Library/CloudStorage/OneDrive-UniversityCollegeLondon/MSci Bahler lab/S.-Pombe-biofilm/Bence folder/GWAS/hail-20221116-1844-0.2.104-1940d9e8eaab.log


In [29]:

hl.import_vcf(snps,reference_genome=my_reference_genome,array_elements_required=False).write('data.mt', overwrite=True)
mt = hl.read_matrix_table('data.mt')
print('Samples: %d  Variants: %d' % (mt.count_cols(), mt.count_rows()))
table = hl.import_table('/Users/bencekover/Library/CloudStorage/OneDrive-UniversityCollegeLondon/MSci Bahler lab/S.-Pombe-biofilm/Bence folder/GWAS/phenotype_gwas.txt', key="strain", no_header=False, delimiter=",",types = {"strain":hl.tstr,"ratio":hl.tfloat64})
mt = mt.annotate_cols(pheno = table[mt.s])
mt = hl.variant_qc(mt)
mt = mt.filter_rows(mt.variant_qc.AF[1] > 0.05)
#mt = mt.filter_rows(mt.variant_qc.p_value_hwe > 1e-4)
print('Samples: %d  Variants: %d' % (mt.count_cols(), mt.count_rows()))

2022-11-10 11:26:28 Hail: INFO: wrote matrix table with 235249 rows and 57 columns in 4 partitions to data.mt


Samples: 57  Variants: 235249


2022-11-10 11:27:03 Hail: INFO: Reading table without type imputation0 + 1) / 1]
  Loading field 'strain' as type str (user-supplied)
  Loading field 'ratio' as type float64 (user-supplied)


Samples: 57  Variants: 136768


In [30]:
eigenvalues, pcs, _ = hl.hwe_normalized_pca(mt.GT)
mt = mt.annotate_cols(scores = pcs[mt.s].scores)
gwas = hl.linear_regression_rows(
    y=mt.pheno.ratio,
    x=mt.GT.n_alt_alleles(),
    covariates=[1.0, mt.scores[0], mt.scores[1], mt.scores[2], mt.scores[3], mt.scores[4]])

2022-11-10 11:27:12 Hail: INFO: hwe_normalize: found 136768 variants after filtering out monomorphic sites.
2022-11-10 11:27:43 Hail: INFO: pca: running PCA with 10 components... + 3) / 4]
2022-11-10 11:28:12 Hail: WARN: 2 of 57 samples have a missing phenotype or covariate.
2022-11-10 11:28:12 Hail: INFO: linear_regression_rows: running on 55 samples for 1 response variable y,
    with input variable x, and 6 additional covariates...


In [31]:
p = hl.plot.manhattan(gwas.p_value)
show(p)

In [32]:
#confert gwas to pandas dataframe
gwas_snp_pd = gwas.to_pandas()

In [35]:
#sort by p_value ascending
gwas_snp_pd = gwas_snp_pd.sort_values(by=['p_value'])
gwas_snp_pd.head(10)

,locus,alleles,n,sum_x,y_transpose_x,beta,standard_error,t_stat,p_value
133295,III:2228740,"[C, T]",55,6.0,2.267975,0.55837,0.06951,8.032999,0.0
133289,III:2227937,"[T, G]",55,6.0,2.267975,0.55837,0.06951,8.032999,0.0
32612,I:3364859,"[C, T]",55,3.235294,2.112613,0.54646,0.073554,7.429402,0.0
133051,III:2219006,"[A, C]",55,5.0,2.231398,0.47136,0.069086,6.822801,0.0
76848,II:1641409,"[C, A, G]",55,4.0,2.173739,0.458168,0.069648,6.578354,0.0
133311,III:2230713,"[G, A]",55,4.0,2.348044,0.444547,0.069142,6.429509,0.0
34916,I:3585317,"[A, C]",55,42.0,5.746557,-0.366654,0.057374,-6.39064,0.0
100776,II:4296877,"[G, A]",55,4.4,2.394839,0.443424,0.070286,6.308884,0.0
100775,II:4296875,"[T, A]",55,4.4,2.394839,0.443424,0.070286,6.308884,0.0
79856,II:1963577,"[A, G, C]",55,4.0,2.256597,0.437406,0.072169,6.060866,0.0


In [ ]:
p = hl.plot.qq(gwas.p_value)
show(p)

2022-11-09 20:12:14 Hail: INFO: Ordering unsorted dataset with network shuffle4]


In [13]:
hl.import_vcf(indels,reference_genome=my_reference_genome,array_elements_required=False,force_bgz=True).write('data.mt', overwrite=True)
mt = hl.read_matrix_table('data.mt')
print('Samples: %d  Variants: %d' % (mt.count_cols(), mt.count_rows()))
table = hl.import_table('/Users/bencekover/Library/CloudStorage/OneDrive-UniversityCollegeLondon/MSci Bahler lab/S.-Pombe-biofilm/Bence folder/GWAS/phenotype_gwas.txt', key="strain", no_header=False, delimiter=",",types = {"strain":hl.tstr,"ratio":hl.tfloat64})
mt = mt.annotate_cols(pheno = table[mt.s])
mt = hl.variant_qc(mt)
mt = mt.filter_rows(mt.variant_qc.AF[1] > 0.1)
#mt = mt.filter_rows(mt.variant_qc.p_value_hwe > 1e-4)
print('Samples: %d  Variants: %d' % (mt.count_cols(), mt.count_rows()))

2022-11-16 19:41:42 Hail: INFO: scanning VCF for sortedness...
2022-11-16 19:42:58 Hail: INFO: Coerced prefix-sorted VCF, requiring additional sorting within data partitions on each query.
2022-11-16 20:37:33 Hail: INFO: wrote matrix table with 21024 rows and 57 columns in 1 partition to data.mt


Samples: 57  Variants: 21024


2022-11-16 20:38:17 Hail: INFO: Reading table without type imputation1 + 0) / 1]
  Loading field 'strain' as type str (user-supplied)
  Loading field 'ratio' as type float64 (user-supplied)


Samples: 57  Variants: 13079


In [14]:
mt.GT.n_alt_alleles()

<Int32Expression of type int32>

In [37]:
eigenvalues, pcs, _ = hl.hwe_normalized_pca(mt.GT)
mt = mt.annotate_cols(scores = pcs[mt.s].scores)
gwas = hl.linear_regression_rows(
    y=mt.pheno.ratio,
    x=mt.GT.n_alt_alleles(),
    covariates=[1.0, mt.scores[0], mt.scores[1], mt.scores[2],  mt.scores[3], mt.scores[4]])
    

2022-11-10 13:37:40 Hail: INFO: hwe_normalize: found 13079 variants after filtering out monomorphic sites.
2022-11-10 13:38:01 Hail: INFO: pca: running PCA with 10 components... + 1) / 1]
2022-11-10 13:38:18 Hail: WARN: 5 of 57 samples have a missing phenotype or covariate.
2022-11-10 13:38:18 Hail: INFO: linear_regression_rows: running on 52 samples for 1 response variable y,
    with input variable x, and 6 additional covariates...


In [41]:
#sort by p_value ascending
gwas_ind_pd = gwas.sort_values(by=['p_value'])
gwas_ind_pd.head(10)

AttributeError: Table instance has no field, method, or property 'sort_values'
    Did you mean:
        Data field: 'p_value' [row]
        Table property: 'row_value'

In [38]:
p = hl.plot.manhattan(gwas.p_value)
show(p)

In [39]:
p = hl.plot.qq(gwas.p_value)
show(p)

2022-11-10 13:38:45 Hail: INFO: Ordering unsorted dataset with network shuffle1]


In [22]:
#confert gwas to pandas dataframe
gwas_ind_pd = gwas.to_pandas()

In [11]:
mtpan = mt.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
    'pheno': struct {
        ratio: float64
    }
    'scores': array<float64>
----------------------------------------
Row fields:
    'locus': locus<pombe>
    'alleles': array<str>
    'rsid': str
    'qual': float64
    'filters': set<str>
    'info': struct {
        AF: array<float64>, 
        BaseQRankSum: float64, 
        DP: int32, 
        DS: bool, 
        Dels: float64, 
        FS: float64, 
        HaplotypeScore: float64, 
        InbreedingCoeff: float64, 
        MLEAC: array<int32>, 
        MLEAF: array<float64>, 
        MQ: float64, 
        MQ0: int32, 
        MQRankSum: float64, 
        QD: float64, 
        RPA: array<int32>, 
        RU: str, 
        ReadPosRankSum: float64, 
        SB: array<float64>, 
        STR: bool, 
        SF: array<str>, 
        AC: array<int32>, 
        AN: int32, 
        `MM.RNA.CPC`: array<

In [17]:
mt["alleles"]

<ArrayExpression of type array<str>>

In [23]:
#gwas_snp_pd find p_value less than 10^-6
#concat gwas_ind_pd[gwas_ind_pd['p_value'] < 1e-4] and gwas_snp_pd[gwas_snp_pd['p_value'] < 1e-6]
snp_hits = gwas_snp_pd[gwas_snp_pd['p_value'] < 1e-7]
indel_hits = gwas_ind_pd[gwas_ind_pd['p_value'] < 1e-4]
#add a column saying snp or indel
snp_hits['type'] = 'snp'
indel_hits['type'] = 'indel'
#concat the two dataframes
hits = pd.concat([snp_hits,indel_hits])
#export results to csv
hits.to_csv('gwas_hits.csv')
hits


/var/folders/mb/vdj4y7t13zd3tzlk99b8ycd40000gn/T/ipykernel_58768/504590319.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  snp_hits['type'] = 'snp'
/var/folders/mb/vdj4y7t13zd3tzlk99b8ycd40000gn/T/ipykernel_58768/504590319.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  indel_hits['type'] = 'indel'


,locus,alleles,n,sum_x,y_transpose_x,beta,standard_error,t_stat,p_value,type
32612,I:3364859,"[C, T]",55,3.235294,2.112613,0.54646,0.073554,7.429402,0.0,snp
34916,I:3585317,"[A, C]",55,42.0,5.746557,-0.366654,0.057374,-6.39064,0.0,snp
76848,II:1641409,"[C, A, G]",55,4.0,2.173739,0.458168,0.069648,6.578354,0.0,snp
100775,II:4296875,"[T, A]",55,4.4,2.394839,0.443424,0.070286,6.308884,0.0,snp
100776,II:4296877,"[G, A]",55,4.4,2.394839,0.443424,0.070286,6.308884,0.0,snp
133051,III:2219006,"[A, C]",55,5.0,2.231398,0.47136,0.069086,6.822801,0.0,snp
133289,III:2227937,"[T, G]",55,6.0,2.267975,0.55837,0.06951,8.032999,0.0,snp
133295,III:2228740,"[C, T]",55,6.0,2.267975,0.55837,0.06951,8.032999,0.0,snp
133311,III:2230713,"[G, A]",55,4.0,2.348044,0.444547,0.069142,6.429509,0.0,snp
1011,I:498798,"[TG, T]",52,6.0,2.530626,0.311554,0.072372,4.304919,0.000089,indel


In [8]:
#look at vcf snps in different samples with scikit allel
import allel
import numpy as np
data = allel.read_vcf(snps,fields=['samples','variants/CHROM','variants/POS','calldata/GT'])
data


{'samples': array(['JB1110', 'JB1117', 'JB1154', 'JB1171', 'JB1174', 'JB1180',
        'JB1197', 'JB1205', 'JB1206', 'JB1207', 'JB22', 'JB4', 'JB758',
        'JB760', 'JB762', 'JB837', 'JB838', 'JB840', 'JB841', 'JB842',
        'JB845', 'JB846', 'JB848', 'JB852', 'JB853', 'JB854', 'JB858',
        'JB862', 'JB864', 'JB869', 'JB870', 'JB871', 'JB872', 'JB873',
        'JB874', 'JB875', 'JB878', 'JB879', 'JB884', 'JB899', 'JB900',
        'JB902', 'JB910', 'JB913', 'JB914', 'JB916', 'JB917', 'JB918',
        'JB929', 'JB930', 'JB931', 'JB934', 'JB938', 'JB939', 'JB942',
        'JB943', 'JB953'], dtype=object),
 'calldata/GT': array([[[ 0, -1],
         [ 0, -1],
         [ 0, -1],
         ...,
         [ 0, -1],
         [ 0, -1],
         [ 0, -1]],
 
        [[ 0, -1],
         [ 0, -1],
         [ 0, -1],
         ...,
         [ 0, -1],
         [ 0, -1],
         [ 0, -1]],
 
        [[ 0, -1],
         [ 0, -1],
         [ 0, -1],
         ...,
         [ 0, -1],
         [ 0, 

In [20]:
#whih variants'POS is 3585317
np.where(data['variants/POS'] == 3585317)

(array([59273]),)

In [33]:
#whih variants'POS is 3585317
np.where(data['variants/POS'] == 2219006)

(array([229318]),)

In [28]:
samples[data['calldata/GT'][59273,:,0] == 1]

array(['JB1110', 'JB1117', 'JB1154', 'JB1171', 'JB1180', 'JB1197',
       'JB1205', 'JB1206', 'JB1207', 'JB4', 'JB758', 'JB762', 'JB837',
       'JB838', 'JB840', 'JB841', 'JB842', 'JB845', 'JB846', 'JB848',
       'JB852', 'JB853', 'JB854', 'JB858', 'JB862', 'JB871', 'JB873',
       'JB874', 'JB875', 'JB878', 'JB884', 'JB899', 'JB900', 'JB902',
       'JB910', 'JB913', 'JB916', 'JB917', 'JB918', 'JB929', 'JB939',
       'JB942', 'JB943', 'JB953'], dtype=object)

In [23]:
#which 2228740	
np.where(data['variants/POS'] == 2228740)

(array([229691]),)

In [9]:
samples = data['samples']
#index of JB914
index = np.where(samples == 'JB914')[0][0]
index


44

In [19]:
data['calldata/GT'].shape

(235249, 57, 2)

In [24]:

data['calldata/GT'][229691,44,:]

array([ 1, -1], dtype=int8)

In [25]:
#at snp 100775, which entry is 1
data['calldata/GT'][229691,:,0]

array([0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], dtype=int8)

In [34]:
data['calldata/GT'][229318,:,0]

array([0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], dtype=int8)

In [26]:
#show me the name of strains that have the previous mutation
samples[data['calldata/GT'][229691,:,0] == 1]

array(['JB1205', 'JB1206', 'JB1207', 'JB872', 'JB914', 'JB934'],
      dtype=object)

In [35]:
data['calldata/GT'][133295,44,:]

array([ 0, -1], dtype=int8)

In [36]:
data['calldata/GT'][34916,44,:]

array([ 0, -1], dtype=int8)

In [12]:
callset = allel.read_vcf(snps)

In [13]:
sorted(callset.keys())

['calldata/GT',
 'samples',
 'variants/ALT',
 'variants/CHROM',
 'variants/FILTER_PASS',
 'variants/ID',
 'variants/POS',
 'variants/QUAL',
 'variants/REF']

In [14]:
callset['samples']

array(['JB1110', 'JB1117', 'JB1154', 'JB1171', 'JB1174', 'JB1180',
       'JB1197', 'JB1205', 'JB1206', 'JB1207', 'JB22', 'JB4', 'JB758',
       'JB760', 'JB762', 'JB837', 'JB838', 'JB840', 'JB841', 'JB842',
       'JB845', 'JB846', 'JB848', 'JB852', 'JB853', 'JB854', 'JB858',
       'JB862', 'JB864', 'JB869', 'JB870', 'JB871', 'JB872', 'JB873',
       'JB874', 'JB875', 'JB878', 'JB879', 'JB884', 'JB899', 'JB900',
       'JB902', 'JB910', 'JB913', 'JB914', 'JB916', 'JB917', 'JB918',
       'JB929', 'JB930', 'JB931', 'JB934', 'JB938', 'JB939', 'JB942',
       'JB943', 'JB953'], dtype=object)

In [ ]:

full_snps = pd.read_csv('/Users/bencekover/Library/CloudStorage/OneDrive-UniversityCollegeLondon/MSci Bahler lab/S.-Pombe-biofilm/Bence folder/QTL mapping/SupplementaryDataset_S7_genotype.tsv', sep='\t',header=15)

In [6]:
table.show()

""
"# Supplementary dataset S8: Tab delimited file containing the genotype information of the segregant of the library for 4,481 markers."
str
"""#"""
"""# The file is tab-delimited."""
"""# First 2 fields are the position of the marker"""
"""# 3rd field is the 968 allele sequence"""
"""# 4th field is the Y0036 allele sequence"""
"""# next fields are genotypes of parents and segregants for the markers:"""
"""# 0: 968, 1: Y0036, NA: missing data."""
"""# """


In [11]:

hl.import_vcf('/Users/bencekover/Downloads/genomeGenerator/testData/test.vcf',reference_genome=my_reference_genome).write('data.mt', overwrite=True)
mt = hl.read_matrix_table('data.mt')
print('Samples: %d  Variants: %d' % (mt.count_cols(), mt.count_rows()))

FatalError: InvalidHeader: Your input file has a malformed header: We never saw a header line specifying VCF version

Java stack trace:
htsjdk.tribble.TribbleException$InvalidHeader: Your input file has a malformed header: We never saw a header line specifying VCF version
	at htsjdk.variant.vcf.VCFCodec.readActualHeader(VCFCodec.java:108)
	at htsjdk.tribble.AsciiFeatureCodec.readHeader(AsciiFeatureCodec.java:79)
	at is.hail.io.vcf.LoadVCF$.parseHeader(LoadVCF.scala:1255)
	at is.hail.io.vcf.MatrixVCFReader$.apply(LoadVCF.scala:1595)
	at is.hail.io.vcf.MatrixVCFReader$.fromJValue(LoadVCF.scala:1668)
	at is.hail.expr.ir.MatrixReader$.fromJson(MatrixIR.scala:89)
	at is.hail.expr.ir.IRParser$.matrix_ir_1(Parser.scala:1803)
	at is.hail.expr.ir.IRParser$.$anonfun$matrix_ir$1(Parser.scala:1719)
	at is.hail.utils.StackSafe$More.advance(StackSafe.scala:64)
	at is.hail.utils.StackSafe$.run(StackSafe.scala:16)
	at is.hail.utils.StackSafe$StackFrame.run(StackSafe.scala:32)
	at is.hail.expr.ir.IRPar